In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [68]:
total = pd.read_csv("../CSV/train_data.csv")
test = pd.read_csv("../CSV/pred_test.csv")

print("total shape : {}, test shape : {}".format(total.shape, test.shape))

total shape : (368088, 12), test shape : (15120, 12)


In [69]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [70]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

# "year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"
features = ["month", "hour", "구분_int", "기온(°C)"]
# features = ["year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"]
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

X = total[features]
y = total["공급량"]

In [71]:
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)
print(test.shape)

(306768, 4) (306768,)
(61320, 4) (61320,)
(15120, 12)


In [72]:
model = Sequential()
model.add(Dense(units = 16, input_dim = 4, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 16)                80        
                                                                 
 dense_32 (Dense)            (None, 32)                544       
                                                                 
 dense_33 (Dense)            (None, 64)                2112      
                                                                 
 dense_34 (Dense)            (None, 32)                2080      
                                                                 
 dense_35 (Dense)            (None, 16)                528       
                                                                 
 dense_36 (Dense)            (None, 1)                 17        
                                                                 
Total params: 5,361
Trainable params: 5,361
Non-traina

In [73]:
model.compile(loss = "mean_squared_error", 
              optimizer = "rmsprop", 
              metrics = ["accuracy"])

early_stopping = EarlyStopping(patience = 30)

hist = model.fit(train_x, train_y, 
                 epochs = 100, 
                 batch_size = 50, 
                 validation_data = (val_x, val_y), 
                 callbacks = [early_stopping])

Epoch 1/100
6136/6136 [==============================] - 6s 833us/step - loss: 521324.9062 - accuracy: 0.0000e+00 - val_loss: 518692.7500 - val_accuracy: 0.0000e+00
Epoch 2/100
6136/6136 [==============================] - 5s 894us/step - loss: 122050.7656 - accuracy: 0.0000e+00 - val_loss: 88668.7500 - val_accuracy: 0.0000e+00
Epoch 3/100
6136/6136 [==============================] - 6s 964us/step - loss: 41742.8438 - accuracy: 0.0000e+00 - val_loss: 43978.3750 - val_accuracy: 0.0000e+00
Epoch 4/100
6136/6136 [==============================] - 6s 936us/step - loss: 36431.3398 - accuracy: 0.0000e+00 - val_loss: 53010.5352 - val_accuracy: 0.0000e+00
Epoch 5/100
6136/6136 [==============================] - 5s 844us/step - loss: 33722.8398 - accuracy: 0.0000e+00 - val_loss: 36924.4727 - val_accuracy: 0.0000e+00
Epoch 6/100
6136/6136 [==============================] - 5s 855us/step - loss: 31515.2090 - accuracy: 0.0000e+00 - val_loss: 55643.7656 - val_accuracy: 0.0000e+00
Epoch 7/100
6136/61

Epoch 51/100
6136/6136 [==============================] - 5s 746us/step - loss: 22323.3516 - accuracy: 0.0000e+00 - val_loss: 29640.3008 - val_accuracy: 0.0000e+00
Epoch 52/100
6136/6136 [==============================] - 5s 738us/step - loss: 22372.4922 - accuracy: 0.0000e+00 - val_loss: 44803.2266 - val_accuracy: 0.0000e+00
Epoch 53/100
6136/6136 [==============================] - 5s 745us/step - loss: 22267.8789 - accuracy: 0.0000e+00 - val_loss: 38005.5195 - val_accuracy: 0.0000e+00
Epoch 54/100
6136/6136 [==============================] - 5s 738us/step - loss: 22247.2227 - accuracy: 0.0000e+00 - val_loss: 62854.2930 - val_accuracy: 0.0000e+00
Epoch 55/100
6136/6136 [==============================] - 5s 744us/step - loss: 22328.5781 - accuracy: 0.0000e+00 - val_loss: 31353.6602 - val_accuracy: 0.0000e+00
Epoch 56/100
6136/6136 [==============================] - 5s 742us/step - loss: 22345.8848 - accuracy: 0.0000e+00 - val_loss: 31088.0801 - val_accuracy: 0.0000e+00
Epoch 57/100
613

In [74]:
model.evaluate(val_x, val_y)

1917/1917 [==============================] - 1s 523us/step - loss: 34718.5352 - accuracy: 0.0000e+00


[34718.53515625, 0.0]

In [75]:
pred = model.predict(val_x)

true_y = pd.DataFrame({"y" : val_y})
pred_y = pd.DataFrame(pred)

true_y.reset_index(drop = True, inplace = True)
true_y.reset_index(inplace = True)
pred_y.reset_index(inplace = True)

print("NMAE : {:.6f}".format(nmae(true_y, pred_y)))

NMAE : 0.534333


In [76]:
pred_y

,index,0
0,0,2064.705322
1,1,1770.634521
2,2,1637.093262
3,3,1706.222168
4,4,1847.824219
...,...,...
61315,61315,635.831787
61316,61316,624.263184
61317,61317,616.861450
61318,61318,584.593628


In [77]:
true_y

,index,y
0,0,1765.008
1,1,1679.186
2,2,1610.885
3,3,1604.123
4,4,1711.506
...,...,...
61315,61315,681.033
61316,61316,669.961
61317,61317,657.941
61318,61318,610.953


In [78]:
submission = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/sample_submission.csv')

In [79]:
test_x = test[features]
preds = model.predict(test_x)
submission['공급량'] = preds
submission.to_csv("../CSV/Submission/23_4_features_DeepLearning_model.csv", index = False)